In [1]:
from keras.applications import VGG16
import tensorflow as tf
import keras
import glob
import numpy as np
import pandas as pd
import os
import shutil 
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
import os,cv2
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
from keras.utils import np_utils
from keras.callbacks import EarlyStopping

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split,KFold
from keras.utils import to_categorical
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve

%matplotlib inline

Using TensorFlow backend.


In [ ]:
import imutils
from scipy import ndimage as ndi
from skimage.feature import canny
from skimage.filters import roberts, farid, sobel, scharr, prewitt, laplace
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import area_closing, disk, dilation, binary_erosion, remove_small_objects, erosion, opening, closing, reconstruction, binary_closing

In [2]:
## Define path ##
PATH = os.getcwd()
data_path = PATH + '/CT'
data_dir_list = os.listdir(data_path)

IMG_WIDTH=224
IMG_HEIGHT=224

IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)
num_channel=1
num_classes = 2
labels_name={'Covid':0,'non-Covid':1}

img_data_list=[]
labels_list = []

## Retrieval of images and labels
for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
    label_data = labels_name[dataset]
    for img in img_list:
        im2= cv2.imread(data_path + '/'+ dataset + '/'+ img , 0)
        im= cv2.imread(data_path + '/'+ dataset + '/'+ img , 0)
        ret1,th1 = cv2.threshold(im, 127,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        h,w = im.shape[0:2]
        mask = np.zeros(im.shape, dtype=np.uint8)

        left = int(np.round(h/(2)))
        right = int(np.round(h/(2)))
        for y in range(left-int(h/5.2), left+int(h/3.9)):
            for x in range(0, 4):
                im[y][x] = 255

        for y in range(right-int(h/5.2), right+int(h/3.9)):
                for x in range(w-4, w):
                    im[y][x] = 255
        
        ret1,im2 = cv2.threshold(im2,180,180, cv2.THRESH_TOZERO)   
        ## Binary image ##
        binary = im2 < 150

        ## Remove the blobs connected to the border of the image ##
        cleared = clear_border(binary)

        ## Label image ##
        label_image = label(cleared)


        ## Keep the labels with 2 largest areas ##
        areas = [r.area for r in regionprops(label_image)]
        areas.sort()


        if len(areas) > 2:
            for region in regionprops(label_image):
                if region.area <= 5:
                    for coordinates in region.coords: 
                        label_image[coordinates[0], coordinates[1]] = 0
                        mask[coordinates[0], coordinates[1]] = 255


        large_binary = label_image > 0  

        ## Erosion operation with a disk of radius 2. This operation is seperate the lung nodules attached to the blood vessels ##
        selem = disk(0)
        binary_eroded = binary_erosion(large_binary, selem)


        ## Closure operation with a disk of radius 10. This operation is to keep nodules attached to the lung wall ##
        selem = disk(9)
        binary_closed = binary_closing(binary_eroded, selem)

        start_point = (0, 0) 
        end_point = (0,h) 
        color = (0,0,0) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point, end_point, color, 5)

        start_point2 = (w, 0) 
        end_point2 = (w,h) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)

        start_point2 = (0, h) 
        end_point2 = (w,h) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)

        start_point2 = (0, 0) 
        end_point2 = (w,0) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)


        ## Fill in the small holes inside the binary mask of lungs ##
        edges = roberts(binary_closed)
        hole_closed = ndi.binary_fill_holes(edges)

        ## Superimpose the binary mask on the input image. ##
        ZERO_VALUE = 0
        get_high_vals = (hole_closed == 0)
        im[get_high_vals] = ZERO_VALUE # minimum value
        
                
        input_img_resize=cv2.resize(im,(224,224))
        img_data_list.append(input_img_resize)
        labels_list.append(label_data) 

Loaded the images of dataset-Covid

Loaded the images of dataset-non-Covid



In [3]:
## Image normalization
img_data = np.array(img_data_list)
img_data = img_data.astype('float32')
img_data /= 255
print (img_data.shape)

(2280, 224, 224, 3)


In [ ]:
if num_channel==1:
    if K.common.image_dim_ordering()=='th':
        img_data= np.expand_dims(img_data, axis=1) 
#         moment_data= np.expand_dims(moment_data, axis=1) 
        print (img_data.shape)
#         print (moment_data.shape)
    else:
        img_data= np.expand_dims(img_data, axis=3) 
#         moment_data= np.expand_dims(moment_data, axis=3) 
        print (img_data.shape)
#         print (moment_data.shape)

else:
    if K.image_dim_ordering()=='th':
        img_data=np.rollaxis(img_data,3,1)
        print (img_data.shape)

In [4]:
# convert class labels to on-hot encoding
labels = np.array(labels_list)
print(np.unique(labels,return_counts=True))
Y = np_utils.to_categorical(labels, num_classes)

(array([0, 1]), array([1152, 1128], dtype=int64))


In [5]:
#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=4)
X_train.shape

(2052, 224, 224, 3)

In [ ]:
## Convert train and test set to RGB
X_train = tf.convert_to_tensor(X_train)
X_train = tf.image.grayscale_to_rgb(X_train)

X_test = tf.convert_to_tensor(X_test)
X_test = tf.image.grayscale_to_rgb(X_test)

proto_tensor_val = tf.make_tensor_proto(X_train)
X_train = tf.make_ndarray(proto_tensor_val)

proto_tensor_val = tf.make_tensor_proto(X_test)
X_test = tf.make_ndarray(proto_tensor_val)

In [16]:
### VGG16 initialization ###
from keras.models import Model
# include top should be False to remove the softmax layer
pretrained_model = VGG16()
pretrained_model = Model(inputs=pretrained_model.inputs, outputs=pretrained_model.layers[-2].output)
pretrained_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0   

In [19]:
## Feature extraction on train and val set

vgg_features_train = pretrained_model.predict(X_train)
vgg_features_val = pretrained_model.predict(X_test)

vgg_features_train = vgg_features_train.reshape(vgg_features_train.shape[0], vgg_features_train.shape[1] ,1)
vgg_features_val = vgg_features_val.reshape(vgg_features_val.shape[0], vgg_features_val.shape[1] ,1)

vgg_features_train.shape

(2052, 4096, 1)

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, InputLayer, BatchNormalization, Dropout

## Fully Connected Layer ##
model2 = Sequential()
model2.add(Flatten(input_shape=(4096,1)))
model2.add(Dense(100, activation='relu'))
model2.add(BatchNormalization())
model2.add(Dropout(0.5))
model2.add(Dense(2, activation='softmax'))

# compile the model
model2.compile(optimizer=keras.optimizers.adam(lr=0.00004, decay= 1e-5), metrics=['accuracy'], loss='binary_crossentropy')

model2.summary()

# train model using features generated from VGG16 model
hist = model2.fit(vgg_features_train, y_train, epochs=100, batch_size=128, validation_data=(vgg_features_val, y_test))


In [33]:
## Saving training output to csv file

hist_df = pd.DataFrame(hist.history) 
hist_csv_file = 'history/test_VGG_history+Segmentation.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

history = pd.read_csv("history/test_VGG_history+Segmentation.csv")
val_loss = history['val_loss'].to_numpy()
val_acc = history['val_accuracy'].to_numpy()
loss = history['loss'].to_numpy()
acc = history['accuracy'].to_numpy()


In [ ]:
## Plot Acc vs Loss graph ##
plt.plot(acc)
plt.plot(loss)
plt.plot(val_acc)
plt.plot(val_loss)
plt.title('Training Accuracy vs Loss (VGG16+Segmentation)', fontsize=20,)
plt.yticks(fontsize=20)
plt.ylabel('Accuracy (%)', fontsize=20)
plt.xticks(fontsize=20)
plt.xlabel('Epoch',fontsize=20)
plt.legend(['Train_acc', 'Train_loss', 'Val_acc', 'Val_loss'], loc='best', fontsize=22)
plt.gcf().set_size_inches(15,10) 
#plt.savefig('C:/Users/CooL/Desktop/Result/VGG/VGG_train_graph.jpg')
plt.show()

In [ ]:
################ Testing phase VGG16 ################

PATH = os.getcwd()
data_path = PATH + '/Testing_Set'
data_dir_list = os.listdir(data_path)

import seaborn as sns
import matplotlib.pyplot as plt   
from sklearn.metrics import confusion_matrix, roc_curve

test_list = []
y_pred = []
labels_name ={'Covid-19':0,'Non-Covid-19':1}


for dataset in data_dir_list:
    img_list=os.listdir(data_path+'/'+ dataset)
    label_data = labels_name[dataset]
    for img in img_list:
        im= cv2.imread(data_path + '/'+ dataset + '/'+ img , 0)
        ret1,th1 = cv2.threshold(im, 127,255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        h,w = im.shape[0:2]
        mask = np.zeros(im.shape, dtype=np.uint8)

        left = int(np.round(h/(2)))
        right = int(np.round(h/(2)))
        for y in range(left-int(h/5.2), left+int(h/3.9)):
            for x in range(0, 4):
                im[y][x] = 255

        for y in range(right-int(h/5.2), right+int(h/3.9)):
                for x in range(w-4, w):
                    im[y][x] = 255

        ## Binary image ##
        binary = im < 150

        ## Remove the blobs connected to the border of the image ##
        cleared = clear_border(binary)

        ## Label image ##
        label_image = label(cleared)


        ## Keep the labels with 2 largest areas ##
        areas = [r.area for r in regionprops(label_image)]
        areas.sort()


        if len(areas) > 2:
            for region in regionprops(label_image):
                if region.area <= 5:
                    for coordinates in region.coords: 
                        label_image[coordinates[0], coordinates[1]] = 0
                        mask[coordinates[0], coordinates[1]] = 255


        large_binary = label_image > 0  

        ## Erosion operation with a disk of radius 2. This operation is seperate the lung nodules attached to the blood vessels ##
        selem = disk(0)
        binary_eroded = binary_erosion(large_binary, selem)


        ## Closure operation with a disk of radius 10. This operation is to keep nodules attached to the lung wall ##
        selem = disk(10)
        binary_closed = binary_closing(binary_eroded, selem)

        start_point = (0, 0) 
        end_point = (0,h) 
        color = (0,0,0) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point, end_point, color, 5)

        start_point2 = (w, 0) 
        end_point2 = (w,h) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)

        start_point2 = (0, h) 
        end_point2 = (w,h) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)

        start_point2 = (0, 0) 
        end_point2 = (w,0) 
        binary_closed = cv2.line(np.float32(binary_closed), start_point2, end_point2, color, 5)


        ## Fill in the small holes inside the binary mask of lungs ##
        edges = roberts(binary_closed)
        hole_closed = ndi.binary_fill_holes(edges)

        ## Superimpose the binary mask on the input image. ##
        ZERO_VALUE = 0
        get_high_vals = (hole_closed == 0)
        im[get_high_vals] = ZERO_VALUE # minimum value
        
                
        input_img_resize=cv2.resize(im,(224,224))
        test_list.append(input_img_resize)
        y_pred.append(label_data)

test_image = np.array(test_list)
test_image = test_image.astype('float32')
test_image /= 255

if num_channel==1:
    if K.common.image_dim_ordering()=='th':
        test_image= np.expand_dims(test_image, axis=1) 
        print (img_data.shape)
    else:
        test_image= np.expand_dims(test_image, axis=3) 
        #print (test_image.shape)

else:
    if K.image_dim_ordering()=='th':
        test_image=np.rollaxis(test_image,3,1)
        print (test_image.shape)
    else:
        test_image= np.expand_dims(test_image, axis=0)
        print (test_image.shape)
        
## Convert train and test set to RGB
test_image = tf.convert_to_tensor(test_image)
test_image = tf.image.grayscale_to_rgb(test_image)

proto_tensor_val = tf.make_tensor_proto(test_image)
test_image = tf.make_ndarray(proto_tensor_val)

## Feature extraction on test images ##
test_features = pretrained_model.predict(test_image)


test_features = test_features.reshape(200,4096,1)
predict_list = []
predict_1 = model2.predict_proba(test_features)

fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_pred, predict_1[:,1])

from sklearn.metrics import auc
import pandas as pd
auc_keras = auc(fpr_keras, tpr_keras)
optimum = thresholds_keras[np.argmax(tpr_keras - fpr_keras)]
print('Optimal cut off point = ',optimum)
print('AUC = ', auc_keras)

gmeans = np.sqrt(tpr_keras * (1-fpr_keras))
ix = np.argmax(gmeans)
optimal = thresholds_keras[ix]
print('Optimal cut off point = ',thresholds_keras[ix])

x = optimal # Best => x = 0.98892

# for j in range(0,1):
#     for i in range(0, len(predict_1)):
#         if predict_1[i][1] >= x:
#             predict_list.append(1)
#         else:
#             predict_list.append(0)
#     print(confusion_matrix(y_pred, predict_list), 'x =',"%.5f" % x, '\n')

cf_matrix=confusion_matrix(y_pred, predict_list)
tp = cf_matrix[0][0]
fn = cf_matrix[0][1]
fp = cf_matrix[1][0]
tn = cf_matrix[1][1]

TrueP = '{0:.2%}'.format(tp/(tp+fn))
FalseN = '{0:.2%}'.format(fn/(tp+fn))
FalseP = '{0:.2%}'.format(fp/(fp+tn))
TrueN = '{0:.2%}'.format(tn/(fp+tn))
print(TrueP, FalseN, FalseP, TrueN)

xy_Labels = ['Covid', 'non-Covid']
group_names = ['True Positive', 'False Negative', 'False Positive','True Negative']
group_counts = ['{0:0.0f}'.format(value) for value in cf_matrix.flatten()]
group_percentage = [TrueP, FalseN, FalseP, TrueN]
labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentage)]
labels = np.asarray(labels).reshape(2,2)

### Confusion Matrix VGG16+Segmentation ###
# plt.title('(VGG-16), Threshold = '+str("{:.4f}".format(optimal)), fontsize = 20)
plt.title('Confusion Matrix (VGG-16)',fontsize = 18)
sns.heatmap(cf_matrix, fmt='', annot=labels, annot_kws={'size':18} ,xticklabels=xy_Labels, yticklabels=xy_Labels, cmap='Blues')
plt.xlabel('Predicted label', fontsize = 15) # x-axis label with fontsize 15
plt.ylabel('True label', fontsize = 15) # y-axis label with fontsize 15
plt.show()

### ROC Curves VGG16+Segmentation ###
plt.figure(figsize=(6,6))
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
plt.scatter(fpr_keras[ix],tpr_keras[ix], marker='o', color='black', label='Optimal Threshold = {:.4f})'.format(optimal))
plt.xlabel('False positive rate (FPR)', fontsize=15)
plt.ylabel('True positive rate (TPR)', fontsize=15)
plt.title('ROC (VGG-16)',fontsize=22)
plt.legend(loc='best', fontsize=15)
plt.show()
